# Explainable fraud detection model

In this example we develop a small fraud detection model for credit card transactions based on XGBoost, export it to TorchScript using Hummingbird (https://github.com/microsoft/hummingbird) and run Shapley Value Sampling explanations (see https://captum.ai/api/shapley_value_sampling.html for reference) on it, also exported to TorchScript.

We load both the original model and the explainability model in RedisAI and trigger them in a DAG.

## Data

For this example we use a dataset of transactions made by credit cards in September 2013 by European cardholders. 
The dataset presents transactions that occurred in two days, with 492 frauds out of 284,807 transactions.

The dataset is available at https://www.kaggle.com/mlg-ulb/creditcardfraud. For anonymity purposes, the features are 28 PCA features (V1 to V28), along with transaction Time and Amount.

__In order to run this notebook please download the `creditcard.csv` file from Kaggle and place it in the `data/` directory.__

Once the file is in place, we start by importing Pandas and reading the data. We create a dataframe of covariates and a dataframe of targets.

In [45]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/creditcard.csv')

In [2]:
X = df.drop(['Class'], axis=1)
Y = df['Class']

## Model

We start off by randomly splitting train and test datasets.

In [3]:
from sklearn.model_selection import train_test_split

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

Next we use XGBoost to classify the transactions. Note that we convert the arguments to `fit` to NumPy arrays.

In [4]:
from xgboost import XGBClassifier

model = XGBClassifier(label_encoder=False)
model.fit(X_train.to_numpy(), y_train.to_numpy())

/home/dvirdukhan/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:32:38] WARNING: ../src/learner.cc:573: 
Parameters: { "label_encoder" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:32:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              label_encoder=False, learning_rate=0.300000012, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

We now obtain predictions on the test dataset and binarize the output probabilities to get a target.

In [5]:
y_pred = model.predict(X_test.to_numpy())
predictions = [round(value) for value in y_pred]

We evaluate the accuracy of our model on the test set (this is just an example: the dataset is heavily unbalanced so accuracy is not a fair characterization in this case).

In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.96%


Looking at the confusion matrix gives a clearer representation.

In [7]:
confusion_matrix(y_test, predictions)

array([[93813,     8],
       [   28,   138]])

We are interested to explore are casesof fraud, so we extract them from the test set.

In [8]:
X_test_fraud = X_test[y_test == 1].to_numpy()

We verify how many times we are getting it right.

In [9]:
model.predict(X_test_fraud) == 1

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True, False,  True,
       False,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True, False,  True, False,  True, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True,  True,  True,
        True, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True,  True,

## Exporting to TorchScript with Hummingbird

From the project page (https://github.com/microsoft/hummingbird):

> Hummingbird is a library for compiling trained traditional ML models into tensor computations. Hummingbird allows users to seamlessly leverage neural network frameworks (such as PyTorch) to accelerate traditional ML models.

Hummingbird can take scikit-learn, XGBoost or LightGBM models and export them to PyTorch, TorchScript, ONNX and TVM. This works very well for running ML models on RedisAI and take advantage of vectorized CPU instructions or GPU.

We choose to convert the boosted tree to tensor computations using the `gemm` implementation.

In [12]:
from hummingbird.ml import convert, load

In [13]:
extra_config={
     "tree_implementation": "gemm"
}

hummingbird_model = convert(model, 'torchscript', test_input=X_test_fraud, extra_config=extra_config)

At this point, `hm_model` is an object containing a TorchScript model that is ready to be exported.

In [14]:
import torch

torch.jit.save(hummingbird_model.model, "models/fraud_detection_model.pt")

We can verify everything works by loading the model and running a prediction. The model outputs a tuple containing the predicted classes and the output probabilities.

In [15]:
loaded_model = torch.jit.load("models/fraud_detection_model.pt")

X_test_fraud_tensor = torch.from_numpy(X_test_fraud)

loaded_output_classes, loaded_output_probs = loaded_model(X_test_fraud_tensor)

We can now compare against the original output from the XGBoost model.

In [16]:
xgboost_output_classes = torch.from_numpy(model.predict(X_test_fraud))

torch.equal(loaded_output_classes, xgboost_output_classes)

True

## Serving model and explainer in RedisAI

At this point we can load the models we exported into RedisAI and serve them from there. After making sure RedisAI is running, we initialize the client.

In [34]:
import redisai

rai = redisai.Client()

We read the model and the explainer from the saved TorchScript.

In [64]:
with open("models/fraud_detection_model.pt", "rb") as f:
    fraud_detection_model_blob = f.read()

with open("torch_shapely.py", "rb") as f:
    shapely_script = f.read()

We load both models into RedisAI.

In [65]:
rai.modelstore("fraud_detection_model", "TORCH", "CPU", fraud_detection_model_blob)
rai.scriptstore("shapely_script", device='CPU', script=shapely_script, entry_points=["shapely_sample"] )

'OK'

All, set, it's now test time. We reuse our `X_test_fraud` NumPy array we created previously. We set it, run both models, and get predictions and explanations as arrays.

In [66]:
rai.tensorset("fraud_input", X_test_fraud, dtype="float")

rai.scriptexecute("shapely_script", "shapely_sample", inputs = ["fraud_input"], keys = ["fraud_detection_model"], args = ["20", "2", "0"], outputs=["fraud_explanations"])

rai_expl = rai.tensorget("fraud_explanations")

We check whether the winning feature is consistent to what we found earlier.

In [67]:
winning_feature_redisai = np.argmax(rai_expl[0], axis=0)

print("Winning feature: %d" % winning_feature_redisai)



Winning feature: 14


In [71]:
rai_expl[0]

array([ 0.  , -0.05,  0.  ,  0.05,  0.2 ,  0.  ,  0.1 ,  0.  ,  0.  ,
       -0.05,  0.15,  0.  ,  0.1 ,  0.  ,  0.5 ,  0.  ,  0.05, -0.1 ,
        0.  ,  0.  ,  0.05,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ])

Alternatively we can set up a RedisAI DAG and run it in one swoop.

In [74]:
dag = rai.dag(routing ="fraud_detection_model")
dag.tensorset("fraud_input", X_test_fraud, dtype="float")
dag.modelexecute("fraud_detection_model", "fraud_input", ["fraud_pred", "fraud_prob"])
dag.scriptexecute("shapely_script", "shapely_sample", inputs = ["fraud_input"], keys = ["fraud_detection_model"], args = ["20", "2", "0"], outputs=["fraud_explanations"])
dag.tensorget("fraud_pred")
dag.tensorget("fraud_explanations")

We now set the input and request a DAG execution, which will produce the desired outputs.

In [75]:
# rai.tensorset("fraud_input", X_test_fraud, dtype="float")

_, _, _, dag_pred, dag_expl = dag.execute()

In [76]:
dag_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

We can now check that the winning feature matches with what we computed earlier on the first sample in the test batch.

In [77]:
winning_feature_redisai_dag = np.argmax(dag_expl[0])

print("Winning feature: %d" % winning_feature_redisai_dag)

Winning feature: 14
